In [1]:
import os
import pandas as pd
import numpy as np
from tabulate import tabulate
from datetime import datetime
from sklearn.impute import SimpleImputer
from tabula import read_pdf

# utilized for creating models and visualization
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# utilized for metrics on my models
from sklearn.metrics import confusion_matrix, classification_report

import math
import modeling

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('finalcsv.csv', index_col=0)
df

,full_state,state,county,population,revenue_per_person,state_funding,funding,buildvalue,agrivalue,area,...,resl_value,drought_freq,drought_score,drought_loss,hurricane_freq,hurricane_score,hurricane_loss,storm_freq,storm_score,storm_loss
0,Alabama,AL,Autauga,54571,3756,204968676,54697779694,5.075584e+09,21460000.0,594.448312,...,2.77649,31.943949,4.191694,1825.417662,0.056426,2.946097,3272.974160,65.740741,7.631697,2.012527e+05
1,Alabama,AL,Baldwin,182265,3756,684587340,54697779694,2.314295e+10,120383000.0,1589.815853,...,2.73796,18.006075,12.636110,4641.483953,0.208845,12.704112,9185.266739,108.666667,23.224606,1.912266e+06
2,Alabama,AL,Barbour,27457,3756,103128492,54697779694,2.206426e+09,105565000.0,885.001636,...,2.53910,53.439915,18.092328,1555.704677,0.073180,7.311136,615.508974,62.481481,12.471689,1.771194e+05
3,Alabama,AL,Bibb,22915,3756,86068740,54697779694,1.643641e+09,4199000.0,622.460425,...,2.59038,36.989399,2.374610,17459.131710,0.043792,1.884830,2594.946558,70.777778,5.382791,4.353395e+05
4,Alabama,AL,Blount,57322,3756,215301432,54697779694,4.794278e+09,242855000.0,644.821022,...,2.66787,34.517290,11.941503,23440.487786,0.012192,5.013860,17810.245704,77.074074,13.540158,3.887720e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,Wyoming,WY,Sweetwater,43806,5353,234493518,6448803714,4.583896e+09,16454000.0,10426.919825,...,2.56275,110.133225,2.840916,67258.895675,0.000000,0.000000,NaN,10.962963,3.481503,6.589745e+04
3138,Wyoming,WY,Teton,21294,5353,113986782,6448803714,3.476594e+09,17582000.0,3996.855337,...,2.59592,56.280417,2.707191,32886.299165,0.000000,0.000000,NaN,9.481481,6.210718,2.104278e+05
3139,Wyoming,WY,Uinta,21118,5353,113044654,6448803714,2.018844e+09,26662000.0,2081.651840,...,2.62771,67.922428,5.762984,175826.688938,0.000000,0.000000,NaN,12.962963,5.590674,8.755011e+04
3140,Wyoming,WY,Washakie,8533,5353,45677149,6448803714,8.867450e+08,43019000.0,2238.665800,...,2.57422,72.708852,10.952769,223781.785275,0.000000,0.000000,NaN,9.222222,7.378384,3.851772e+04


In [3]:
df = df.rename(columns = {'funding' : 'federal_funding'})

In [4]:
df

,full_state,state,county,population,revenue_per_person,state_funding,federal_funding,buildvalue,agrivalue,area,...,resl_value,drought_freq,drought_score,drought_loss,hurricane_freq,hurricane_score,hurricane_loss,storm_freq,storm_score,storm_loss
0,Alabama,AL,Autauga,54571,3756,204968676,54697779694,5.075584e+09,21460000.0,594.448312,...,2.77649,31.943949,4.191694,1825.417662,0.056426,2.946097,3272.974160,65.740741,7.631697,2.012527e+05
1,Alabama,AL,Baldwin,182265,3756,684587340,54697779694,2.314295e+10,120383000.0,1589.815853,...,2.73796,18.006075,12.636110,4641.483953,0.208845,12.704112,9185.266739,108.666667,23.224606,1.912266e+06
2,Alabama,AL,Barbour,27457,3756,103128492,54697779694,2.206426e+09,105565000.0,885.001636,...,2.53910,53.439915,18.092328,1555.704677,0.073180,7.311136,615.508974,62.481481,12.471689,1.771194e+05
3,Alabama,AL,Bibb,22915,3756,86068740,54697779694,1.643641e+09,4199000.0,622.460425,...,2.59038,36.989399,2.374610,17459.131710,0.043792,1.884830,2594.946558,70.777778,5.382791,4.353395e+05
4,Alabama,AL,Blount,57322,3756,215301432,54697779694,4.794278e+09,242855000.0,644.821022,...,2.66787,34.517290,11.941503,23440.487786,0.012192,5.013860,17810.245704,77.074074,13.540158,3.887720e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,Wyoming,WY,Sweetwater,43806,5353,234493518,6448803714,4.583896e+09,16454000.0,10426.919825,...,2.56275,110.133225,2.840916,67258.895675,0.000000,0.000000,NaN,10.962963,3.481503,6.589745e+04
3138,Wyoming,WY,Teton,21294,5353,113986782,6448803714,3.476594e+09,17582000.0,3996.855337,...,2.59592,56.280417,2.707191,32886.299165,0.000000,0.000000,NaN,9.481481,6.210718,2.104278e+05
3139,Wyoming,WY,Uinta,21118,5353,113044654,6448803714,2.018844e+09,26662000.0,2081.651840,...,2.62771,67.922428,5.762984,175826.688938,0.000000,0.000000,NaN,12.962963,5.590674,8.755011e+04
3140,Wyoming,WY,Washakie,8533,5353,45677149,6448803714,8.867450e+08,43019000.0,2238.665800,...,2.57422,72.708852,10.952769,223781.785275,0.000000,0.000000,NaN,9.222222,7.378384,3.851772e+04


In [5]:
df=df.fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3142 entries, 0 to 3141
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   full_state          3142 non-null   object 
 1   state               3142 non-null   object 
 2   county              3142 non-null   object 
 3   population          3142 non-null   int64  
 4   revenue_per_person  3142 non-null   int64  
 5   state_funding       3142 non-null   int64  
 6   federal_funding     3142 non-null   int64  
 7   buildvalue          3142 non-null   float64
 8   agrivalue           3142 non-null   float64
 9   area                3142 non-null   float64
 10  risk_score          3142 non-null   float64
 11  resl_score          3142 non-null   float64
 12  resl_value          3142 non-null   float64
 13  drought_freq        3142 non-null   float64
 14  drought_score       3142 non-null   float64
 15  drought_loss        3142 non-null   float64
 16  hurric

In [6]:
# creating target variable
df['support_value'] = (df.state_funding + df.federal_funding) - (df.drought_loss + df.hurricane_loss +
                                                                 df.storm_loss)

In [7]:
df

,full_state,state,county,population,revenue_per_person,state_funding,federal_funding,buildvalue,agrivalue,area,...,drought_freq,drought_score,drought_loss,hurricane_freq,hurricane_score,hurricane_loss,storm_freq,storm_score,storm_loss,support_value
0,Alabama,AL,Autauga,54571,3756,204968676,54697779694,5.075584e+09,21460000.0,594.448312,...,31.943949,4.191694,1825.417662,0.056426,2.946097,3272.974160,65.740741,7.631697,2.012527e+05,5.490254e+10
1,Alabama,AL,Baldwin,182265,3756,684587340,54697779694,2.314295e+10,120383000.0,1589.815853,...,18.006075,12.636110,4641.483953,0.208845,12.704112,9185.266739,108.666667,23.224606,1.912266e+06,5.538044e+10
2,Alabama,AL,Barbour,27457,3756,103128492,54697779694,2.206426e+09,105565000.0,885.001636,...,53.439915,18.092328,1555.704677,0.073180,7.311136,615.508974,62.481481,12.471689,1.771194e+05,5.480073e+10
3,Alabama,AL,Bibb,22915,3756,86068740,54697779694,1.643641e+09,4199000.0,622.460425,...,36.989399,2.374610,17459.131710,0.043792,1.884830,2594.946558,70.777778,5.382791,4.353395e+05,5.478339e+10
4,Alabama,AL,Blount,57322,3756,215301432,54697779694,4.794278e+09,242855000.0,644.821022,...,34.517290,11.941503,23440.487786,0.012192,5.013860,17810.245704,77.074074,13.540158,3.887720e+05,5.491265e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,Wyoming,WY,Sweetwater,43806,5353,234493518,6448803714,4.583896e+09,16454000.0,10426.919825,...,110.133225,2.840916,67258.895675,0.000000,0.000000,0.000000,10.962963,3.481503,6.589745e+04,6.683164e+09
3138,Wyoming,WY,Teton,21294,5353,113986782,6448803714,3.476594e+09,17582000.0,3996.855337,...,56.280417,2.707191,32886.299165,0.000000,0.000000,0.000000,9.481481,6.210718,2.104278e+05,6.562547e+09
3139,Wyoming,WY,Uinta,21118,5353,113044654,6448803714,2.018844e+09,26662000.0,2081.651840,...,67.922428,5.762984,175826.688938,0.000000,0.000000,0.000000,12.962963,5.590674,8.755011e+04,6.561585e+09
3140,Wyoming,WY,Washakie,8533,5353,45677149,6448803714,8.867450e+08,43019000.0,2238.665800,...,72.708852,10.952769,223781.785275,0.000000,0.000000,0.000000,9.222222,7.378384,3.851772e+04,6.494219e+09


In [8]:
df.support_value[0]

54902542018.8636

In [9]:
# feature engineering due to findings in exploration
df['pop_density'] = df.population / df.area

In [10]:
df

,full_state,state,county,population,revenue_per_person,state_funding,federal_funding,buildvalue,agrivalue,area,...,drought_score,drought_loss,hurricane_freq,hurricane_score,hurricane_loss,storm_freq,storm_score,storm_loss,support_value,pop_density
0,Alabama,AL,Autauga,54571,3756,204968676,54697779694,5.075584e+09,21460000.0,594.448312,...,4.191694,1825.417662,0.056426,2.946097,3272.974160,65.740741,7.631697,2.012527e+05,5.490254e+10,91.801085
1,Alabama,AL,Baldwin,182265,3756,684587340,54697779694,2.314295e+10,120383000.0,1589.815853,...,12.636110,4641.483953,0.208845,12.704112,9185.266739,108.666667,23.224606,1.912266e+06,5.538044e+10,114.645353
2,Alabama,AL,Barbour,27457,3756,103128492,54697779694,2.206426e+09,105565000.0,885.001636,...,18.092328,1555.704677,0.073180,7.311136,615.508974,62.481481,12.471689,1.771194e+05,5.480073e+10,31.024801
3,Alabama,AL,Bibb,22915,3756,86068740,54697779694,1.643641e+09,4199000.0,622.460425,...,2.374610,17459.131710,0.043792,1.884830,2594.946558,70.777778,5.382791,4.353395e+05,5.478339e+10,36.813585
4,Alabama,AL,Blount,57322,3756,215301432,54697779694,4.794278e+09,242855000.0,644.821022,...,11.941503,23440.487786,0.012192,5.013860,17810.245704,77.074074,13.540158,3.887720e+05,5.491265e+10,88.895985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,Wyoming,WY,Sweetwater,43806,5353,234493518,6448803714,4.583896e+09,16454000.0,10426.919825,...,2.840916,67258.895675,0.000000,0.000000,0.000000,10.962963,3.481503,6.589745e+04,6.683164e+09,4.201241
3138,Wyoming,WY,Teton,21294,5353,113986782,6448803714,3.476594e+09,17582000.0,3996.855337,...,2.707191,32886.299165,0.000000,0.000000,0.000000,9.481481,6.210718,2.104278e+05,6.562547e+09,5.327688
3139,Wyoming,WY,Uinta,21118,5353,113044654,6448803714,2.018844e+09,26662000.0,2081.651840,...,5.762984,175826.688938,0.000000,0.000000,0.000000,12.962963,5.590674,8.755011e+04,6.561585e+09,10.144828
3140,Wyoming,WY,Washakie,8533,5353,45677149,6448803714,8.867450e+08,43019000.0,2238.665800,...,10.952769,223781.785275,0.000000,0.000000,0.000000,9.222222,7.378384,3.851772e+04,6.494219e+09,3.811645


In [11]:
df['support_level'] = pd.cut(df.support_value, 4, labels = ['bottom tier', 'below average', 'above average', 'top tier'])

In [12]:
df

,full_state,state,county,population,revenue_per_person,state_funding,federal_funding,buildvalue,agrivalue,area,...,drought_loss,hurricane_freq,hurricane_score,hurricane_loss,storm_freq,storm_score,storm_loss,support_value,pop_density,support_level
0,Alabama,AL,Autauga,54571,3756,204968676,54697779694,5.075584e+09,21460000.0,594.448312,...,1825.417662,0.056426,2.946097,3272.974160,65.740741,7.631697,2.012527e+05,5.490254e+10,91.801085,bottom tier
1,Alabama,AL,Baldwin,182265,3756,684587340,54697779694,2.314295e+10,120383000.0,1589.815853,...,4641.483953,0.208845,12.704112,9185.266739,108.666667,23.224606,1.912266e+06,5.538044e+10,114.645353,bottom tier
2,Alabama,AL,Barbour,27457,3756,103128492,54697779694,2.206426e+09,105565000.0,885.001636,...,1555.704677,0.073180,7.311136,615.508974,62.481481,12.471689,1.771194e+05,5.480073e+10,31.024801,bottom tier
3,Alabama,AL,Bibb,22915,3756,86068740,54697779694,1.643641e+09,4199000.0,622.460425,...,17459.131710,0.043792,1.884830,2594.946558,70.777778,5.382791,4.353395e+05,5.478339e+10,36.813585,bottom tier
4,Alabama,AL,Blount,57322,3756,215301432,54697779694,4.794278e+09,242855000.0,644.821022,...,23440.487786,0.012192,5.013860,17810.245704,77.074074,13.540158,3.887720e+05,5.491265e+10,88.895985,bottom tier
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,Wyoming,WY,Sweetwater,43806,5353,234493518,6448803714,4.583896e+09,16454000.0,10426.919825,...,67258.895675,0.000000,0.000000,0.000000,10.962963,3.481503,6.589745e+04,6.683164e+09,4.201241,bottom tier
3138,Wyoming,WY,Teton,21294,5353,113986782,6448803714,3.476594e+09,17582000.0,3996.855337,...,32886.299165,0.000000,0.000000,0.000000,9.481481,6.210718,2.104278e+05,6.562547e+09,5.327688,bottom tier
3139,Wyoming,WY,Uinta,21118,5353,113044654,6448803714,2.018844e+09,26662000.0,2081.651840,...,175826.688938,0.000000,0.000000,0.000000,12.962963,5.590674,8.755011e+04,6.561585e+09,10.144828,bottom tier
3140,Wyoming,WY,Washakie,8533,5353,45677149,6448803714,8.867450e+08,43019000.0,2238.665800,...,223781.785275,0.000000,0.000000,0.000000,9.222222,7.378384,3.851772e+04,6.494219e+09,3.811645,bottom tier


In [13]:
df.support_level.value_counts()

bottom tier      2201
below average     500
above average     383
top tier           58
Name: support_level, dtype: int64

In [14]:
X_train, y_train, X_validate, y_validate, X_test, y_test = modeling.train_validate_test(df, 'support_level')

In [15]:
X_train

,revenue_per_person,buildvalue,agrivalue,risk_score,resl_score,resl_value,drought_freq,drought_score,hurricane_freq,hurricane_score,storm_freq,storm_score,pop_density
1616,4471,1.057160e+08,18601000.0,2.814082,54.868002,2.74340,76.222222,7.447481,0.000000,0.000000,13.370370,2.763764,0.752442
2688,4744,2.379925e+09,129518000.0,10.196566,51.741600,2.58708,55.002686,8.610116,0.041439,2.777447,63.370370,13.999974,24.345207
694,6400,7.189950e+09,34713000.0,15.034935,56.230202,2.81151,4.277778,9.018533,0.008800,2.933785,85.444444,19.009341,157.927155
556,4074,3.423221e+09,453144000.0,11.528897,54.598798,2.72994,61.634478,21.553306,0.000000,0.000000,6.000000,9.568270,21.782951
991,5210,3.577340e+08,52550000.0,4.099580,54.382600,2.71913,18.836009,2.632980,0.000000,0.000000,89.777778,7.799481,6.646986
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2309,5554,2.650580e+09,13243000.0,6.575932,57.585402,2.87927,0.000000,0.000000,0.031023,1.587182,26.185185,7.302026,71.169478
366,4047,3.255315e+09,235881000.0,17.947441,49.816398,2.49082,29.781388,25.524370,0.244438,24.909909,114.259259,20.873776,52.017418
1197,6810,1.266408e+10,6322000.0,3.536079,58.106800,2.90534,5.009398,1.686076,0.089715,2.949282,52.666667,5.014600,416.226814
479,4075,2.750301e+09,51278000.0,6.643944,53.275400,2.66377,50.542181,5.234111,0.011913,3.238076,52.814815,10.499221,105.913001


In [16]:
X_train_scaled, X_validate_scaled, X_test_scaled = modeling.scale_data(X_train, X_validate, X_test)

In [17]:
(y_train == 'bottom tier').mean()

0.7117680500284252

In [18]:
# Setting up my baseline accuracy.  Models must have a better accuracy than this established baseline.
print(f'The baseline accuracy for bottom tier counties in all cases within the Climate Risk Assessment dataset is {(y_train == "bottom tier").mean():.2}')

The baseline accuracy for bottom tier counties in all cases within the Climate Risk Assessment dataset is 0.71


In [19]:
# Creating multiple Decision Tree models utilizing the above chosen features.  Created a for loop that allows me
# to look at multiple max depths at once for the train dataset.
for i in range (5, 16, 5):
    clf = DecisionTreeClassifier(max_depth= i, random_state=123)
    clf = clf.fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_train_scaled)
    print(f'Tree with max depth of {i}')
    print(pd.DataFrame(classification_report(y_train, y_pred, output_dict=True)))

Tree with max depth of 5
           above average  below average  bottom tier   top tier  accuracy  \
precision       0.941748       1.000000     0.849897   0.882353  0.866401   
recall          0.923810       0.227273     0.990415   0.909091  0.866401   
f1-score        0.932692       0.370370     0.914792   0.895522  0.866401   
support       210.000000     264.000000  1252.000000  33.000000  0.866401   

             macro avg  weighted avg  
precision     0.918499      0.884000  
recall        0.762647      0.866401  
f1-score      0.778344      0.834858  
support    1759.000000   1759.000000  
Tree with max depth of 10
           above average  below average  bottom tier  top tier  accuracy  \
precision            1.0            1.0          1.0       1.0       1.0   
recall               1.0            1.0          1.0       1.0       1.0   
f1-score             1.0            1.0          1.0       1.0       1.0   
support            210.0          264.0       1252.0      33.0  

In [20]:
for i in range (5, 16, 5):
    y_pred = clf.predict(X_validate_scaled)
    print(f'Tree with max depth of {i}')
    print(pd.DataFrame(classification_report(y_validate, y_pred, output_dict=True)))

Tree with max depth of 5
           above average  below average  bottom tier  top tier  accuracy  \
precision       0.980000       0.984127     0.998069       0.9  0.992042   
recall          0.989899       1.000000     0.992322       0.9  0.992042   
f1-score        0.984925       0.992000     0.995188       0.9  0.992042   
support        99.000000     124.000000   521.000000      10.0  0.992042   

            macro avg  weighted avg  
precision    0.965549      0.992103  
recall       0.970555      0.992042  
f1-score     0.968028      0.992053  
support    754.000000    754.000000  
Tree with max depth of 10
           above average  below average  bottom tier  top tier  accuracy  \
precision       0.980000       0.984127     0.998069       0.9  0.992042   
recall          0.989899       1.000000     0.992322       0.9  0.992042   
f1-score        0.984925       0.992000     0.995188       0.9  0.992042   
support        99.000000     124.000000   521.000000      10.0  0.992042  

In [21]:
# After finding the best model using decision tree methodology I compared the train set's accuracy score with the 
# validate set's accuracy score to determine the level of consistency between trained data and untrained data.
clf = DecisionTreeClassifier(max_depth=5, random_state=123)
clf = clf.fit(X_train_scaled, y_train)
clf.score(X_train_scaled, y_train) - clf.score(X_validate_scaled, y_validate)

0.001679879000456963

In [22]:
# Utilizing my established features within the X_train variable and my target with y_train, I created a logistic
# regression model to best predict churn rate.  
logit = LogisticRegression(random_state=123)
logit = logit.fit(X_train_scaled, y_train)
y_pred = logit.predict(X_train_scaled)
print(pd.DataFrame(classification_report(y_train, y_pred, output_dict=True)))
print()
print(pd.DataFrame(classification_report(y_validate, logit.predict(X_validate_scaled), output_dict=True)))

           above average  below average  bottom tier   top tier  accuracy  \
precision       0.710280            0.0     0.752147   0.950000  0.750995   
recall          0.361905            0.0     0.979233   0.575758  0.750995   
f1-score        0.479495            0.0     0.850798   0.716981  0.750995   
support       210.000000          264.0  1252.000000  33.000000  0.750995   

             macro avg  weighted avg  
precision     0.603107      0.637975  
recall        0.479224      0.750995  
f1-score      0.511819      0.676267  
support    1759.000000   1759.000000  

           above average  below average  bottom tier   top tier  accuracy  \
precision       0.823529            0.0     0.719888   1.000000  0.725464   
recall          0.282828            0.0     0.986564   0.500000  0.725464   
f1-score        0.421053            0.0     0.832389   0.666667  0.725464   
support        99.000000          124.0   521.000000  10.000000  0.725464   

            macro avg  weighted 

In [23]:
modeling.modeling(X_train_scaled, y_train, X_validate_scaled, y_validate, 5, 3, 15)

LogisticRegression(random_state=123)
              
train score: 
               precision    recall  f1-score   support

above average       0.71      0.36      0.48       210
below average       0.00      0.00      0.00       264
  bottom tier       0.75      0.98      0.85      1252
     top tier       0.95      0.58      0.72        33

     accuracy                           0.75      1759
    macro avg       0.60      0.48      0.51      1759
 weighted avg       0.64      0.75      0.68      1759

val score: 
               precision    recall  f1-score   support

above average       0.82      0.28      0.42        99
below average       0.00      0.00      0.00       124
  bottom tier       0.72      0.99      0.83       521
     top tier       1.00      0.50      0.67        10

     accuracy                           0.73       754
    macro avg       0.64      0.44      0.48       754
 weighted avg       0.62      0.73      0.64       754

            
DecisionTreeClassifier(

(LogisticRegression(random_state=123),
 DecisionTreeClassifier(max_depth=5, random_state=123),
 RandomForestClassifier(max_depth=3, random_state=123),
 KNeighborsClassifier(n_neighbors=15))

In [24]:
knn = KNeighborsClassifier(n_neighbors=15, weights='uniform')
knn.fit(X_train_scaled, y_train)
pred_test = knn.predict(X_test_scaled)
print('KNN')
print('')
print('test score: ')
print(classification_report(y_test, pred_test))

KNN

test score: 
               precision    recall  f1-score   support

above average       0.88      0.69      0.77        74
below average       0.90      0.85      0.87       112
  bottom tier       0.91      0.96      0.93       428
     top tier       0.93      0.93      0.93        15

     accuracy                           0.90       629
    macro avg       0.90      0.86      0.88       629
 weighted avg       0.90      0.90      0.90       629

